# Table of Contents

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import requests
from bs4 import BeautifulSoup

# 1. Acquire table and convert to dataframe

In [3]:
raw_website = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(raw_website,features='html.parser')

In [4]:
table_postcodes = soup.find('table',{'class':'wikitable'})
df = pd.read_html(str(table_postcodes))[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Drop rows without a Neighborhood

In [50]:
#Get rid of Postal Codes not tied to a Borough
indicies_to_drop = df[df['Borough']=='Not assigned'].index
df.drop(indicies_to_drop, inplace=True)

#Make sure they were eliminated
df[df['Borough']=='Not assigned']

,Postal Code,Borough,Neighborhood


### No need to concatenate here, just replacing '/' with commas

In [53]:
df['Neighborhood']=df['Neighborhood'].str.replace('/',',', regex=False)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [55]:
print("There are {} postal code rows in the Wikipedia table".format(df.shape[0]))

There are 103 postal code rows in the Wikipedia table


# 2. Composing Location Information

## Adding Latitude and Longitude

In [7]:
import geocoder

#### Geocoder isn't responding for the hardcoded zip , so the CSV file containing the coordinates will be imported instead

In [9]:
df_location = pd.read_csv('Geospatial_Coordinates.csv')
df_location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Postal Code is a bit different in both tables, so original df will be adjusted. After normalized, we will merge on Postal Code

In [57]:
df.columns = ['Postal Code', 'Borough', 'Neighborhood']


df_toronto =pd.merge(df, df_location, on='Postal Code')
df_toronto['Label']= df_toronto['Postal Code'] + ": " + df_toronto['Neighborhood']
df_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Label
0,M3A,North York,Parkwoods,43.753259,-79.329656,M3A: Parkwoods
1,M4A,North York,Victoria Village,43.725882,-79.315572,M4A: Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,"M5A: Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,"M6A: Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,"M7A: Queen's Park , Ontario Provincial Government"


# 3. Exploring Toronto

In [11]:
import folium
from geopy.geocoders import Nominatim

In [12]:
geolocator = Nominatim(user_agent='Coursera_Toronto_Explorer')
location = geolocator.geocode('Toronto, Canada')

In [58]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=10.7)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Postal Code'] + ': ' + df_toronto['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Now let's explore venues within the neighborhoods using Foursquare.

### Here, we are going to see common types of venues within the neighborhoods. This might add characteristics for cluster by.

note that results have been left in, but client key/id has been left out to avoid irresponsible API key use

In [14]:
CLIENT_ID = 'C20I21HEFESYE3NTDRW45EKJGMZP3XDZ4BQNFCZUQLTSEJJI' 
CLIENT_SECRET = 'M2JJR2R1YI3RFZSFAM4HTQRIIXRQLY5PGVD1MDOOOQN2OSOR' 
VERSION = '20180605' # Using version from this course for consistency

In [18]:
#create function for API Calls on all neighborhoods

def getNearbyVenues(names, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, hoods, lat, lng in zip(names, neighborhoods, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            '300')
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            hoods,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                  'Neighborhoods',
                  'Postal Latitude', 
                  'Postal Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names = df_toronto['Postal Code'],
                                 neighborhoods=df_toronto['Neighborhood'],
                                latitudes=df_toronto['Latitude'],
                                longitudes=df_toronto['Longitude'])

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


In [20]:
toronto_venues.head()

,Postal Code,Neighborhoods,Postal Latitude,Postal Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


### Let's take a look at our results and see the different venue categories, total venues, and the venues per neighborhood

In [21]:
print("Total Number of venues: {}\nThere are a total of {} unique venue categories".format(
    toronto_venues.shape[0], 
    len(toronto_venues['Venue Category'].unique())))

Total Number of venues: 2142
There are a total of 270 unique venue categories


In [22]:
#neighorhood counts
toronto_venues.groupby('Postal Code').count()

,Neighborhoods,Postal Latitude,Postal Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,,
M1B,1,1,1,1,1,1,1
M1C,2,2,2,2,2,2,2
M1E,7,7,7,7,7,7,7
M1G,4,4,4,4,4,4,4
M1H,8,8,8,8,8,8,8
M1J,2,2,2,2,2,2,2
M1K,8,8,8,8,8,8,8
M1L,10,10,10,10,10,10,10
M1M,2,2,2,2,2,2,2


### Some neighborhoods have significantly more venues than others. Lets now analyze our results further and massage the data for Clustering

## Analyzing the Neighborhoods

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postal Code column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [24]:
print("Onehot columns = {}\nUnique venues found inoriginal frame = {}".format(
    toronto_onehot.shape[1]-1,
    len(toronto_venues['Venue Category'].unique())))

Onehot columns = 270
Unique venues found inoriginal frame = 270


In [26]:
toronto_grouped = toronto_onehot.groupby('Postal Code').sum().reset_index()
toronto_grouped.head()

,Postal Code,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Soccer Field,Social Club,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [27]:
toronto_grouped.shape

(99, 271)

In [62]:
num_top_venues = 5

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0  Fast Food Restaurant   1.0
1     Accessories Store   0.0
2   Moroccan Restaurant   0.0
3     Martial Arts Dojo   0.0
4        Massage Studio   0.0


----M1C----
                             venue  freq
0                              Bar   1.0
1                      Golf Course   1.0
2                Accessories Store   0.0
3               Mexican Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----M1E----
                 venue  freq
0         Intersection   1.0
1   Mexican Restaurant   1.0
2                 Bank   1.0
3  Rental Car Location   1.0
4    Electronics Store   1.0


----M1G----
                 venue  freq
0          Coffee Shop   2.0
1    Korean Restaurant   1.0
2    Convenience Store   1.0
3    Accessories Store   0.0
4  Monument / Landmark   0.0


----M1H----
                  venue  freq
0                Bakery   1.0
1           Gas Station   1.0
2       Thai Restaurant   1.0
3  Caribbean Restaurant   1.0
4    Ath

                venue  freq
0         Coffee Shop  13.0
1  Italian Restaurant   4.0
2                Café   3.0
3      Sandwich Place   3.0
4     Bubble Tea Shop   2.0


----M5H----
                 venue  freq
0          Coffee Shop   9.0
1                 Café   5.0
2           Restaurant   4.0
3      Thai Restaurant   3.0
4  American Restaurant   3.0


----M5J----
                venue  freq
0         Coffee Shop  12.0
1            Aquarium   5.0
2               Hotel   4.0
3                Café   4.0
4  Italian Restaurant   4.0


----M5K----
                 venue  freq
0          Coffee Shop  11.0
1                Hotel   8.0
2                 Café   7.0
3           Restaurant   5.0
4  Japanese Restaurant   4.0


----M5L----
                 venue  freq
0          Coffee Shop  10.0
1           Restaurant   7.0
2                 Café   7.0
3                Hotel   6.0
4  American Restaurant   4.0


----M5M----
                venue  freq
0  Italian Restaurant   2.0
1         Pizza 

In [31]:
#Write function to sort venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues] 

In [37]:
#Create new df with top 10 venues per neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
1,M1C,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Rental Car Location,Breakfast Spot,Bank,Electronics Store,Intersection,Medical Center,Mexican Restaurant,Doner Restaurant,Distribution Center,Dog Run
3,M1G,Coffee Shop,Indian Restaurant,Korean Restaurant,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,M1H,Hakka Restaurant,Gas Station,Fried Chicken Joint,Bank,Bakery,Athletics & Sports,Thai Restaurant,Caribbean Restaurant,Creperie,Cuban Restaurant


## Cluster Neighborhoods

In [38]:
from sklearn.cluster import KMeans

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0,
       0, 2, 0, 0, 0, 2, 2, 2, 3, 3, 2, 4, 3, 4, 1, 1, 2, 0, 0, 2, 2, 2,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 2, 4, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [42]:
toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

#convert Cluster Labels to int
toronto_merged['Cluster Labels'].apply(pd.to_numeric)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,M3A: Parkwoods,0,Park,Food & Drink Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Falafel Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,M4A: Victoria Village,0,Hockey Arena,Pizza Place,Portuguese Restaurant,Coffee Shop,French Restaurant,Eastern European Restaurant,Dumpling Restaurant,Electronics Store,Drugstore,Dessert Shop
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,"M5A: Regent Park , Harbourfront",2,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Dessert Shop,Bank,French Restaurant
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,"M6A: Lawrence Manor , Lawrence Heights",0,Clothing Store,Women's Store,Furniture / Home Store,Accessories Store,Coffee Shop,Shoe Store,Miscellaneous Shop,Boutique,Event Space,Gift Shop
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,"M7A: Queen's Park , Ontario Provincial Government",4,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Burger Joint,Bar,Beer Bar,Spa,Italian Restaurant,Japanese Restaurant
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,M9A: Islington Avenue,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,"M1B: Malvern , Rouge",0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,M3B: Don Mills,0,Caribbean Restaurant,Gym / Fitness Center,Café,Athletics & Sports,Japanese Restaurant,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Drugstore
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,"M4B: Parkview Hill , Woodbine Gardens",0,Pizza Place,Gym / Fitness Center,Gastropub,Pet Store,Pharmacy,Breakfast Spot,Intersection,Bank,Athletics & Sports,Fast Food Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,"M5B: Garden District, Ryerson",3,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Bubble Tea Shop,Ramen Restaurant,Italian Restaurant


In [40]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Postal Code'], toronto_merged['Cluster Labels']):
    cluster = cluster
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster Examination

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,M3A: Parkwoods,0,Park,Food & Drink Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Falafel Restaurant,Drugstore
1,North York,M4A: Victoria Village,0,Hockey Arena,Pizza Place,Portuguese Restaurant,Coffee Shop,French Restaurant,Eastern European Restaurant,Dumpling Restaurant,Electronics Store,Drugstore,Dessert Shop
3,North York,"M6A: Lawrence Manor , Lawrence Heights",0,Clothing Store,Women's Store,Furniture / Home Store,Accessories Store,Coffee Shop,Shoe Store,Miscellaneous Shop,Boutique,Event Space,Gift Shop
5,Etobicoke,M9A: Islington Avenue,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Scarborough,"M1B: Malvern , Rouge",0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
7,North York,M3B: Don Mills,0,Caribbean Restaurant,Gym / Fitness Center,Café,Athletics & Sports,Japanese Restaurant,Donut Shop,Distribution Center,Dog Run,Doner Restaurant,Drugstore
8,East York,"M4B: Parkview Hill , Woodbine Gardens",0,Pizza Place,Gym / Fitness Center,Gastropub,Pet Store,Pharmacy,Breakfast Spot,Intersection,Bank,Athletics & Sports,Fast Food Restaurant
10,North York,M6B: Glencairn,0,Park,Pizza Place,Pub,Japanese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
11,Etobicoke,"M9B: West Deane Park , Princess Gardens , Mart...",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Scarborough,"M1C: Rouge Hill , Port Union , Highland Creek",0,Construction & Landscaping,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,Downtown Toronto,"M5K: Toronto Dominion Centre , Design Exchange",1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Sporting Goods Shop,Steakhouse
48,Downtown Toronto,"M5L: Commerce Court , Victoria Hotel",1,Coffee Shop,Café,Restaurant,Hotel,American Restaurant,Gym,Japanese Restaurant,Italian Restaurant,Seafood Restaurant,Deli / Bodega
92,Downtown Toronto,M5W: Stn A PO Boxes,1,Coffee Shop,Italian Restaurant,Seafood Restaurant,Café,Cocktail Bar,Hotel,Restaurant,Beer Bar,Japanese Restaurant,Cheese Shop
97,Downtown Toronto,"M5X: First Canadian Place , Underground city",1,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,Deli / Bodega,Steakhouse,Seafood Restaurant,American Restaurant


In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,"M5A: Regent Park , Harbourfront",2,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Café,Theater,Dessert Shop,Bank,French Restaurant
20,Downtown Toronto,M5E: Berczy Park,2,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Italian Restaurant,Cheese Shop,Farmers Market,Beer Bar,Seafood Restaurant,Café
23,East York,M4G: Leaside,2,Sporting Goods Shop,Coffee Shop,Burger Joint,Bank,Furniture / Home Store,Bike Shop,Sports Bar,Fish & Chips Shop,Beer Store,Supermarket
37,West Toronto,"M6J: Little Portugal , Trinity",2,Bar,Restaurant,Vietnamese Restaurant,Café,Vegetarian / Vegan Restaurant,Men's Store,Asian Restaurant,New American Restaurant,Boutique,Brewery
41,East Toronto,"M4K: The Danforth West , Riverdale",2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Liquor Store,Spa,Japanese Restaurant,Juice Bar
43,West Toronto,"M6K: Brockton , Parkdale Village , Exhibition ...",2,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
54,East Toronto,M4M: Studio District,2,Café,Coffee Shop,Bakery,Brewery,Gastropub,American Restaurant,Cheese Shop,Bank,Clothing Store,Stationery Store
55,North York,"M5M: Bedford Park , Lawrence Manor East",2,Coffee Shop,Italian Restaurant,Sandwich Place,Restaurant,Pub,Café,Sushi Restaurant,Liquor Store,Pizza Place,Pharmacy
59,North York,M2N: Willowdale,2,Ramen Restaurant,Sushi Restaurant,Sandwich Place,Restaurant,Café,Pizza Place,Coffee Shop,Indonesian Restaurant,Hotel,Steakhouse
74,Central Toronto,"M5R: The Annex , North Midtown , Yorkville",2,Café,Sandwich Place,Coffee Shop,Liquor Store,Indian Restaurant,Pub,Middle Eastern Restaurant,BBQ Joint,Donut Shop,Pizza Place


In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,"M5B: Garden District, Ryerson",3,Clothing Store,Coffee Shop,Café,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Restaurant,Bubble Tea Shop,Ramen Restaurant,Italian Restaurant
15,Downtown Toronto,M5C: St. James Town,3,Café,Coffee Shop,Cocktail Bar,Italian Restaurant,Gastropub,American Restaurant,Lingerie Store,Farmers Market,Seafood Restaurant,Restaurant
30,Downtown Toronto,"M5H: Richmond , Adelaide , King",3,Coffee Shop,Café,Restaurant,Deli / Bodega,Thai Restaurant,Hotel,American Restaurant,Clothing Store,Gym,Bakery
33,North York,"M2J: Fairview , Henry Farm , Oriole",3,Clothing Store,Coffee Shop,Fast Food Restaurant,Women's Store,Bank,Tea Room,Bakery,Restaurant,Food Court,Japanese Restaurant


In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,"M7A: Queen's Park , Ontario Provincial Government",4,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Burger Joint,Bar,Beer Bar,Spa,Italian Restaurant,Japanese Restaurant
24,Downtown Toronto,M5G: Central Bay Street,4,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Fried Chicken Joint
36,Downtown Toronto,"M5J: Harbourfront East , Union Station , Toron...",4,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Scenic Lookout,Restaurant,Brewery,Sporting Goods Shop


#### We can see the suburban areas are clustered together. As we get closer to the center of the metropolitan area, clusters start to separate based on Coffee Shop occurrence, Shopping, and Restaurant type similarities